In [ ]:
import censusdata, pandas as pd

def import_censusdata(year, variables):
    census_data = censusdata.download('acs5', year, 
                                censusdata.censusgeo([('state','*'), ('county', '*')]), variables)

    dictionary = {'B02001_001E':'Total Pop Estimate', 'B01001_006E':'Males 15-17',\
                  'B01001A_006E':'White Males 15-17','B01001B_006E':'Black Males 15-17', 'B01001I_006E':'Hispanic Males 15-17',\
                   'B19001_001E':'HHI', 'B19001_016E':'HHI 150K-200K','B19001A_001E':'HHI 220K+'}
    census_data.rename(columns =dictionary, inplace=True)
    
    
    for index, data in census_data.iterrows():
        string_county = index.name.split(',')
        census_data.loc[index,'County'], census_data.loc[index,'State']= string_county[0], string_county[1]

       # print(string_county)
    census_data.reset_index(drop=True,inplace=True)
    census_dataframe = pd.DataFrame(census_data)

    return census_dataframe.groupby(['State', 'County']).agg({'Total Pop Estimate':'sum',
                                                                'Males 15-17':'sum', 'Black Males 15-17':'sum', 'Hispanic Males 15-17':'sum',\
                                                            'HHI 220K+':'sum'})

males_census_2019df = import_censusdata(2019, ['B02001_001E', 'B01001_006E','B01001A_006E','B01001_006E' , 'B01001B_006E', 'B01001I_006E',\
                                                'B19001_001E', 'B19001_016E','B19001A_001E'])
# males_census_2019df.sort_values(by = 'Males 15-17', ascending=False)

In [ ]:
import censusdata, pandas as pd

def import_censusdata0(year, variables, geo_agg):
    census_data = censusdata.download('acs5', year, 
                                censusdata.censusgeo([('state','*'), geo_agg]), variables)

    dictionary = {'B02001_001E': 'Total Pop Estimate '+str(year), 'B19001_001E':'HHI', 'B19001_016E':'HHI 150K-200K','B19001A_001E':'HHI 220K+',\
    'B01001_006E':'Males 15-17', 'B01001A_006E':'White Males 15-17','B01001B_006E':'Black Males 15-17', 'B01001I_006E':'Hispanic Males 15-17',\
    'B01001_030E': 'Females 15-17', 'B01001A_021E':'White Females 15-17',' B01001B_021E':'Black Females 15-17','B01001I_021E': 'Hispanic Females 15-17'}
    
    census_data.rename(columns =dictionary, inplace=True)
    for index, data in census_data.iterrows():
        census_data.loc[index,'County'], census_data.loc[index,'State']= index.name, index.params()[0][1]

    #census_data.reset_index(drop=True,inplace=True)
    census_dataframe = pd.DataFrame(census_data)

    return census_dataframe#.groupby(['State', 'County']).agg({'Total Pop Estimate'+str(year):'sum',
                                                           #     'Males 15-17':'sum', 'Black Males 15-17':'sum', 'Hispanic Males 15-17':'sum',\
                                                           # 'HHI 220K+':'sum'})

male_vars = ['B02001_001E', 'B01001_006E','B01001A_006E','B01001_006E' , 'B01001B_006E', 'B01001I_006E',\
                                                'B19001_001E', 'B19001_016E','B19001A_001E']

female_vars = ['B02001_001E', 'B01001_030E', 'B01001A_021E','B01001B_021E','B01001I_021E']

males_census_2019df = import_censusdata0(2019, male_vars, ('county', '*') )
males_census_2018df = import_censusdata0(2018, male_vars, ('county', '*') )

growth_trend = pd.concat([males_census_2019df['Total Pop Estimate 2019'], males_census_2018df['Total Pop Estimate 2018']], axis=1)
growth_trend['% delta'] = growth_trend['Total Pop Estimate 2019'] / growth_trend['Total Pop Estimate 2018']

In [ ]:
#this variant uses groupings for the variables
import requests
 
class census_api:
    baseurl = "https://api.census.gov/data/2019/acs/acs5?get=NAME,group(B02001)&for=zip%20code%20tabulation%20area:*&in=state:*"
    
    def __init__(self):
        #self.paramx = {'dataset'}
        self.resp = requests.get(self.baseurl).json()
         
    def output(self):
        return self.resp
census_header = census_api().output()

In [25]:
def school_data(year, dataset, var, grade, fips):
    import urllib, json, pandas as pd
    base_url = "https://educationdata.urban.org/api/v1/schools/"
    if (dataset, var) in {('ccd', 'directory'), ('crdc', 'directory'), ('crdc', 'school-finance')}:
         url = base_url+dataset+"/"+var+"/"+str(year)+"/?fips="+str(fips)
    elif (dataset, var) in {('ccd', 'enrollment'), ('crdc', 'retention')}:
         url = base_url+dataset+"/"+var+"/"+str(year)+"/"+str(grade)+"/race/sex/?fips="+str(fips)
    elif (dataset, var) in {('crdc', 'enrollment'), ('crdc', 'sat-act-participation')}:
         url = base_url+dataset+"/"+var+"/"+str(year)+"/race/sex/?fips="+str(fips)

    print(url)
    df_lst = []
    while True:
         try:
               response = urllib.request.urlopen(url)
               retention = json.loads(response.read())
               df = pd.json_normalize(retention, record_path= ['results'])
               if response.getcode() == 200:
                   break
         except Exception as inst:
               print (inst)
    df_lst.append(df)
    while retention['next'] != None:
         url = retention['next']
         print(url)
         while True:
               try:
                    response = urllib.request.urlopen(url)
                    retention = json.loads(response.read())
                    df = pd.json_normalize(retention, record_path= ['results'])
                    if response.getcode() == 200:
                         break
               except Exception as inst:
                    print (inst)
         df_lst.append(df)
    return pd.concat(df_lst)
# school_data(2015, 'ccd', 'enrollment', 'grade-12', 45)

In [26]:
def get_ccd_multiple(lst, var, grade, fips):
    df_loop=[]
    for y in lst:
        d = school_data(y, 'ccd', var, grade, fips)
        df_loop.append(d)
    return df_loop
years = [2015, 2016, 2017, 2018, 2019]
#dfs_ccd_directory = get_ccd_multiple(years, 'directory', False, 45)
#dfs_ccd_enrollment = get_ccd_multiple(years, 'enrollment', 'grade-12', 45)

In [27]:
# The crdc enrollment data is published biennially!!! (The most recent years are 2013, 2015 and 2017)
def get_crdc_multiple(lst, var, grade, fips):
    df_loop=[]
    for y in lst:
        d = school_data(y, 'crdc', var, grade, fips)
        df_loop.append(d)
    return df_loop
years = [2013, 2015, 2017]
#dfs_crdc_directory = get_crdc_multiple(years, 'directory', False, 45)
#dfs_crdc_enrollment = get_crdc_multiple(years, 'enrollment', 'grade-12', 45)
#dfs_crdc_sat_act_part = get_crdc_multiple(years, 'sat-act-participation', 'grade-12', 45)
#dfs_school_finance = get_crdc_multiple(years, 'school-finance', False, 45)
#dfs_retention = get_crdc_multiple(years, 'retention', 'grade-12', 45)

https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=2
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=3
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=4
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=5
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=6
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=7
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=8
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page=9
https://educationdata.urban.org/api/v1/schools/crdc/retention/2013/grade-12/race/sex/?fips=45&page

In [ ]:
i = 2015
for df in dfs_ccd_directory:
    df.to_csv("./educationdata/ccd_directory_" + str(i) + ".csv")
    i+=1

In [ ]:
i = 2015
for df in dfs_ccd_enrollment:
    df.to_csv("./educationdata/ccd_enrollment_" + str(i) + ".csv")
    i+=1

In [ ]:
i = 2013
for df in dfs_crdc_directory:
    df.to_csv("./educationdata/crdc_directory_" + str(i) + ".csv")
    i+=2

In [ ]:
i = 2013
for df in dfs_crdc_enrollment:
    df.to_csv("./educationdata/crdc_enrollment_" + str(i) + ".csv")
    i+=2

In [ ]:
i = 2013
for df in dfs_crdc_sat_act_part:
    df.to_csv("./educationdata/crdc_sat_act_part_" + str(i) + ".csv")
    i+=2

In [ ]:
i = 2013
for df in dfs_school_finance:
    df.to_csv("./educationdata/crdc_school_finance_" + str(i) + ".csv")
    i+=2

In [28]:
i = 2013
for df in dfs_retention:
    df.to_csv("./educationdata/crdc_retention_" + str(i) + ".csv")
    i+=2